In [ ]:
!pip -q install h5py
!pip -q install typing-extensions
!pip -q install wheel
!pip -q install imgaug==0.2.5
!pip -q install malaya
!pip -q install tensorflow==2.9.0
!pip -q install tensorflow_addons

     |████████████████████████████████| 562 kB 13.3 MB/s 
     |████████████████████████████████| 2.8 MB 12.3 MB/s 
     |████████████████████████████████| 235 kB 66.4 MB/s 
     |████████████████████████████████| 1.6 MB 51.6 MB/s 
     |████████████████████████████████| 292 kB 70.8 MB/s 
     |████████████████████████████████| 5.5 MB 50.2 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 1.3 MB 49.0 MB/s 
     |████████████████████████████████| 163 kB 60.8 MB/s 
     |████████████████████████████████| 749 kB 54.8 MB/s 
     |████████████████████████████████| 41 kB 156 kB/s 
     |████████████████████████████████| 7.6 MB 51.5 MB/s 
     |████████████████████████████████| 511.7 MB 3.8 kB/s 
     |████████████████████████████████| 1.1 MB 10.5 MB/s 


In [ ]:
import malaya
import tensorflow as tf
import pandas as pd
import random
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/resource_loader.py:110: UserWarning: You are currently using TensorFlow 2.9.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.10.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops by setting environment variable `TF_ADDONS_PY_OPS=1` or using `tfa.options.disable_custom_kernel()` in your code. To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would need a TensorFlow version equal to or above 2.10.0 and strictly below 2.11.0.
 Note that nightly versions of TensorFlow, as well as non-pip TensorFlow li

True

In [ ]:
pos_model = malaya.pos.transformer(model = 'bert')

Downloading:   0%|          | 0.00/447M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/569k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

In [ ]:
mal_noun = ["NOUN","PROPN"]
mal_pron = ["PRON"]
mal_verb = ["ADX","VERB"]
mal_adv = ["ADV"]
mal_adj = ["ADJ"]
mal_adp = ["ADP"]
mal_conj = ["CCONJ","SCONJ"]
mal_det = ["DET"]
mal_par = ["PART"]
mal_num = ["NUM"]
mal_sym = ["SYM"]
mal_oth = ["X"]

eng_noun = ["NN","NNP","NNPS","NNS"]
eng_pron = ["PRP","PRP$","WP","WP$"]
eng_verb = ["VB","VBD","VBG","VBN","VBP","VBZ","MD"]
eng_adv = ["RB","RBR","RBS"]
eng_adj = ["JJ","JJR","JJS"]
eng_adp = ["IN"]
eng_conj = ["CC"]
eng_det = ["DT","PDT","TO","EX"]
eng_par = ["RP"]
eng_num = ["CD"]
eng_sym = ["$","(",")",",","--",".",":","SYM","``","''"]
eng_oth = ["FW","LS","POS","UH"]

In [ ]:
def english_pos(word):
                eng = ""
                resulting_model = nltk.pos_tag([word])
                pos = resulting_model[0][1]
                if pos in eng_noun:
                    eng = "NOUN"
                elif pos in eng_pron:
                    eng = "PRON"
                elif pos in eng_verb:
                    eng = "VERB"
                elif pos in eng_adv:
                    eng = "ADVB"
                elif pos in eng_adj:
                    eng = "ADJC"
                elif pos in eng_adp:
                    eng = "ADPN"
                elif pos in eng_conj:
                    eng = "CONJ"
                elif pos in eng_det:
                    eng = "DETR"
                elif pos in eng_par:
                    eng = "PCLE"
                elif pos in eng_num:
                    eng = "NMBR"
                elif pos in eng_sym:
                    eng = "SYMB"
                else:
                    eng = "OTHR"
                return eng

In [ ]:
def malay_pos(word):
                mal = ""
                resulting_model = pos_model.analyze(word)
                if not resulting_model:
                    return mal
                pos = resulting_model[0]["type"]
                if pos in mal_noun:
                    mal = "NOUN"
                elif pos in mal_pron:
                    mal = "PRON"
                elif pos in mal_verb:
                    mal = "VERB"
                elif pos in mal_adv:
                    mal = "ADVB"
                elif pos in mal_adj:
                    mal = "ADJC"
                elif pos in mal_adp:
                    mal = "ADPN"
                elif pos in mal_conj:
                    mal = "CONJ"
                elif pos in mal_det:
                    mal = "DETR"
                elif pos in mal_par:
                    mal = "PCLE"
                elif pos in mal_num:
                    mal = "NMBR"
                elif pos in mal_sym:
                    mal = "SYMB"
                else:
                    mal = "OTHR"
                return mal

In [ ]:
def pos_sentence_generator(text):
    words = text.split()
    malay_flag = False
    english_flag = False
    interjection_flag = False
    other_flag = False
    final = ""
    for word in words:
        if word.startswith("<malay>") and word.endswith("</malay>"):
            final += "<malay>"+malay_pos(word[7:-8])+"</malay>"
        elif word.startswith("<malay>"):
            malay_flag = True
            final += "<malay>"+malay_pos(word[7:])
        elif word.endswith("</malay>"):
            malay_flag = False
            final += malay_pos(word[:-8])+"</malay>"
        elif malay_flag:
            final += malay_pos(word)
        
        elif word.startswith("<english>") and word.endswith("</english>"):
            final += "<english>"+english_pos(word[9:-10])+"</english>"
        elif word.startswith("<english>"):
            english_flag = True
            final += "<english>"+english_pos(word[9:])
        elif word.endswith("</english>"):
            english_flag = False
            final += english_pos(word[:-10])+"</english>"
        elif english_flag:
            final += english_pos(word)
        
        elif word.startswith("<interjection>") and word.endswith("</interjection>"):
            final += "<english>"+english_pos(word[14:-15])+"</english>"
        elif word.startswith("<interjection>"):
            interjection_flag = True
            final += "<english>"+english_pos(word[14:])
        elif word.endswith("</interjection>"):
            interjection_flag = False
            final += english_pos(word[:-15])+"</english>"
        elif interjection_flag:
            final += english_pos(word)
        
        elif word.startswith("<other_language>") and word.endswith("</other_language>"):
            final += "<other_language>"+"FN_LG"+"</other_language>"
        elif word.startswith("<other_language>"):
            malay_flag = True
            final += "<other_language>"+"FN_LG"
        elif word.endswith("</other_language>"):
            malay_flag = False
            final += "FN_LG" + "</other_language>"
        elif malay_flag:
            final += "FN_LG"
        
        else:
            final += english_pos(word)
        
        final += " "
    
    return final

In [ ]:
with open("CS_Lang_Data.txt", "r") as f:
    text = f.readlines()

with open("POS_CS_Lang_Data.txt", "w") as f:
    final_txt = ""
    for line in text:
        line = line.strip()
        if line:
            final_txt += pos_sentence_generator(line) +"\n"
        else:
            final_txt += "\n"
    f.write(final_txt)